In [ ]:
# Setup: topic modeling & embeddings
from datasets import load_dataset
import pandas as pd
from sentence_transformers import SentenceTransformer
from src.utils import set_seed

set_seed(42)

# Load dataset
print("Loading dataset FutureMa/EvasionBench...")
ds = load_dataset("FutureMa/EvasionBench")
if isinstance(ds, dict):
    ds = ds[list(ds.keys())[0]]
df = ds.to_pandas()
print("Dataset shape:", df.shape)

# Load lightweight sentence transformer for similarity
embedder = SentenceTransformer("all-MiniLM-L6-v2")
print("Embedder loaded: all-MiniLM-L6-v2")

# Example embedding
example = df.loc[0, "question"] + " " + df.loc[0, "answer"]
print("Sample text:\n", example[:200])

# 03 - Q&A Interaction Analysis

**Objectives:**
- Topic modeling on questions and answers
- Semantic similarity and question-type analysis
- Build answer strategy taxonomy